# Business Problem
## Company sales brochure generator

### - create a product that can generate marketing brochures about a company
####    - for prospective client
####    - for investors
####   - for recruitment

### - the technology
####   - use openai api
####   - use one-shot prompting
####   - stream back results & show with formatting

In [49]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import json

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [5]:
# initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
if api_key and api_key[:8]=='sk-proj-':
    print("API key looks good so far.")
else:
    print("There might be a problem with your API key?")

API key looks good so far.


In [70]:
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [58]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [59]:
ed = Website("https://edwarddonner.com")

In [60]:
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of pr

In [61]:
print(ed.links)

['https://edwarddonner.com/', 'https://edwarddonner.com/outsmart/', 'https://edwarddonner.com/about-me-and-about-nebula/', 'https://edwarddonner.com/posts/', 'https://edwarddonner.com/', 'https://news.ycombinator.com', 'https://nebula.io/?utm_source=ed&utm_medium=referral', 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html', 'https://patents.google.com/patent/US20210049536A1/', 'https://www.linkedin.com/in/eddonner/', 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/', 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/', 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/', 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/', 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/', 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/', 'https://edwarddonner.com/2024/10/16/from-software-engineer-to

In [73]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += """
{
    "links": [
    {"type":"about page", "url":"https://full.url/goes/here/about"},
    {"type":"careers page", "url":"https://another.full.url/careers"}
    ]
}
"""

In [74]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, responsd with \
    full https URL in JSON format. Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [75]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, responsd with     full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https:

In [76]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system", "content":link_system_prompt},
            {"role":"user", "content":get_links_user_prompt(website)}
        ],
        response_format={"type":"json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [78]:
get_links("https://antropic.com")

{'links': ['https://antropic.com/']}

In [79]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/inference-providers',
 '/deepseek-ai/DeepSeek-R1',
 '/deepseek-ai/Janus-Pro-7B',
 '/mistralai/Mistral-Small-24B-Instruct-2501',
 '/deepseek-ai/DeepSeek-V3',
 '/unsloth/DeepSeek-R1-GGUF',
 '/models',
 '/spaces/deepseek-ai/Janus-Pro-7B',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/Qwen/Qwen2.5-Max-Demo',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/Mistral-AI-Game-Jam/NeuralJam',
 '/spaces',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets/cognitivecomputations/dolphin-r1',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/ServiceNow-AI/R1-Distill-SFT',
 '/datasets/bespokelabs/Bespoke-Stratos-17k',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/micros

In [80]:
get_links("https://huggingface.co")

{'links': ['https://huggingface.co/',
  'https://huggingface.co/models',
  'https://huggingface.co/datasets',
  'https://huggingface.co/spaces',
  'https://huggingface.co/enterprise',
  'https://huggingface.co/pricing',
  'https://huggingface.co/docs',
  'https://huggingface.co/blog',
  'https://huggingface.co/join',
  'https://apply.workable.com/huggingface/',
  'https://github.com/huggingface',
  'https://twitter.com/huggingface',
  'https://www.linkedin.com/company/huggingface/']}

In [85]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    # for link in links["links"]:
    #     # result += f"\n\n{link['type']}\n"
    #     result += Website(link["url"]).get_contents()
    return result

In [86]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': ['https://huggingface.co/', 'https://huggingface.co/enterprise', 'https://huggingface.co/pricing', 'https://huggingface.co/docs', 'https://huggingface.co/blog', 'https://huggingface.co/join', 'https://github.com/huggingface', 'https://twitter.com/huggingface', 'https://www.linkedin.com/company/huggingface/']}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Welcome to Inference Providers on the Hub 🔥
smolagents - a smol library to build great agents
Use models from the HF Hub in LM Studio
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Models
deepseek-ai/DeepSeek-R1
Updated
4 days ago
•
1.04M
•
6.47k
deepseek-ai/Janus-Pro-7B
Updated
3 days ago
•
175k
•
2.51k
mistralai/Mistral-Small-24B-Instruct-2501
Updated
2 days ago